# Experiment
Recreate resutts from emargin article with AdaBoost and AdaBoostV instead of arc-gv.  (add computation of emargin later)

http://colt2008.cs.helsinki.fi/papers/08-Wang.pdf page 11

extend to do cross validation

In [1]:
# Import data sets. 
import ipython_notebook_importer
import DataSet as ds

In [6]:
# All features is normalized to [0, 1]? Sum to 1, or just live in this area?
# THe ioanosphere dataset has an dimension where max is 0, so now i try max of abs

# Five fold cross-validation, use this as test error??!?!?!?
# decision stumps, decision trees with depth 1? Yes.
''' 
    We use a finite set of possible decision
    stumps. Specifically, for each feature we consider 100
    thresholds uniformly distributed on [0, 1]. Therefore the
    size of the base classifier set is 2 × 100 × k, where k
    denotes the number of features.
'''
from sklearn.model_selection import KFold
import ipython_notebook_importer
import AdaBoost as ada
import AdaBoostV as adav # importing notebooks might take a few seconds. 
import numpy as np

def experiment(name):
    # Load dataset
    data = ds.DataSet(name)
    X = data.X
    y = data.y
    
    print(name.upper()+ "  ", "\tTest (CV) Error\tMinimum Margin")
    
    kf = KFold(n_splits=3, random_state=None, shuffle=False)
    
    error_ada  = []
    error_adav = []
    mm_ada     = []
    mm_adav    = []
    
    error_ada_s = []
    error_adav_s = []
    error_ada_s_el = []
    mm_ada_s = []
    mm_adav_s = []
    
    error_retrain = []
    mm_retrain = []
    error_retrain_v = []
    mm_retrain_v = []
    
    i = 0
    for train_index, test_index in kf.split(X):
        print(i)
        i+=1
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        # AdaBoost exp
        a = ada.AdaBoost()
        a.fit(X, y, 500)
        error_ada.append(1-a.score(X_test, y_test))
        mm_ada.append(a.minimal_margin(X_train, y_train))
        
        # AdaBoostV experiment
        av = adav.AdaBoostV()
        av.fit(X, y, 500)
        error_adav.append(1-av.score(X_test, y_test))
        mm_adav.append(av.minimal_margin(X_train, y_train))
        
        # AdaBoost; sparsify and select by min margin
        a_s = ada.AdaBoost()
        a_s.force_silent = True
        a_s.fit(X, y, 100) # just to 256 and we won on a single dataset!
        a_s = a_s.sparsify(X, y, 10, "mm")
        error_ada_s.append(1-a_s.score(X_test, y_test))
        mm_ada_s.append(a_s.minimal_margin(X_train, y_train))

        # AdaBoost; sparsify and select by minimal margin
        a_s = ada.AdaBoost()
        a_s.force_silent = True
        a_s.fit(X, y, 100) # just to 256 and we won on a single dataset!
        a_s = a_s.sparsify(X, y, 10, "exploss")
        error_ada_s_el.append(1-a_s.score(X_test, y_test))
        #mm_ada_s_el.append(a_s.minimal_margin(X_train, y_train))
        
        # AdaBoostV; sparsify, retrain and select by minimal margin
        av_s = adav.AdaBoostV()
        av_s.force_silent = True
        av_s.fit(X, y, 100) # just to 256 and we won on a single dataset!
        av_s = av_s.sparsify(X, y, 10)
        error_adav_s.append(1-av_s.score(X_test, y_test))
        mm_adav_s.append(av_s.minimal_margin(X_train, y_train))
        
        #retrain simple
        a_retrain = ada.AdaBoost()
        a_retrain.force_silent = True
        a_retrain = a_retrain.retrain(X, y, 3, max=50)
        error_retrain.append(1-a_retrain.score(X_test, y_test))
        mm_retrain.append(a_retrain.minimal_margin(X_train, y_train))
        
        a_retrain = adav.AdaBoostV()
        a_retrain.force_silent = True
        a_retrain = a_retrain.retrain(X, y, 3, max=50)
        error_retrain_v.append(1-a_retrain.score(X_test, y_test))
        mm_retrain_v.append(a_retrain.minimal_margin(X_train, y_train))
        
        
    a_error_cv = np.mean(error_ada)
    av_error_cv = np.mean(error_adav)
    a_mm_cv  = np.mean(mm_ada)
    av_mm_cv = np.mean(mm_adav)
    
    a_error_cv_s = np.mean(error_ada_s)
    av_error_cv_s = np.mean(error_adav_s)
    a_mm_cv_s  = np.mean(mm_ada_s)
    av_mm_cv_s = np.mean(mm_adav_s)
    
    a_error_cv_s_el = np.mean(error_ada_s_el)
    
    error_retrain = np.mean(error_retrain)
    mm_retrain = np.mean(mm_retrain)
    error_retrain_v = np.mean(error_retrain_v)
    mm_retrain_v = np.mean(mm_retrain_v)
    
    print("AdaBoost: \t", round(a_error_cv,5), "\t", round(a_mm_cv,5))
    print("AdaBoostV: \t", round(av_error_cv, 5), "\t", round(av_mm_cv,5))
    print("---")
    print("AdaBoost (S,mm,r1): \t", round(a_error_cv_s,5), "\t", round(a_mm_cv_s,5))
    print("AdaBoost (S,el,r1): \t", round(a_error_cv_s_el,5))
    print("AdaBoostV (S,mm,r1): \t", round(av_error_cv_s, 5), "\t", round(av_mm_cv_s,5))
    print("Retrain: \t", round(error_retrain, 5), "\t", round(mm_retrain,5))
    print("RetrainV: \t", round(error_retrain_v, 5), "\t", round(mm_retrain_v,5))

    
    print("")
# repeat train sparsify on 10->100; 

In [ ]:
for name in ds.DataSet.names: 
    experiment(name)

DIABETES   	Test (CV) Error	Minimum Margin
0
